<a href="https://colab.research.google.com/github/teacherSsamko/DL-study/blob/main/w7_2_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [7주차] 심화과제: 자신만의 text 파일로 LLM Instruction-tuning 해보기

앞부분에서 충전해둔 크레딧이 아직 많이남아서 EC2를 사용하는 대신 notebook으로 진행했습니다.

In [1]:
!pip install torch wandb datasets evaluate transformers huggingface_hub trl pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import sys
import math
import torch
import wandb
import logging
import argparse
import evaluate
import datasets
import transformers
import pandas as pd
from sklearn.model_selection import train_test_split

from typing import Optional
from itertools import chain
from dataclasses import dataclass, field

from datasets import Dataset, DatasetDict
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
)
from transformers.trainer_utils import get_last_checkpoint
from huggingface_hub import login
from google.colab import userdata
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

wandb.init(project="Hanghae99")
wandb.run.name = "gpt-finetuning-support"

HF_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

login(token=HF_TOKEN)

torch.cuda.empty_cache()


@dataclass
class Arguments:
    model_name_or_path: Optional[str] = field(
        default=None
    )  # HuggingFace hub에서 pre-trained 모델로 사용할 모델의 이름
    torch_dtype: Optional[str] = field(
        default=None, metadata={"choices": ["auto", "bfloat16", "float16", "float32"]}
    )  # 우리 모델의 precision(data type이라고 이해하시면 됩니다)
    block_size: int = field(default=1024)  # Fine-tuning에 사용할 input text의 길이
    num_workers: Optional[int] = field(
        default=None
    )  # Data를 업로드하거나 전처리할 때 사용할 worker 숫자

MODEL_NAME = "facebook/opt-125m"

# ArgumentParser 대신 직접 변수 설정
model_args = Arguments(
    model_name_or_path=MODEL_NAME,  # 사용할 모델 이름
    torch_dtype="float16",  # 데이터 타입
    block_size=256,
    num_workers=2
)

training_args = TrainingArguments(
    output_dir="./results",  # 결과물을 저장할 디렉토리
    num_train_epochs=30,  # 학습 에포크 수
    per_device_train_batch_size=2,  # 배치 사이즈
    weight_decay=0.01,  # 가중치 감쇠
    logging_dir="./logs",  # 로그 저장 위치
    logging_steps=10,
    logging_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    do_eval=True,
    per_device_eval_batch_size=1,
    eval_steps=10,
    evaluation_strategy="steps",
    eval_accumulation_steps=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
)

# args를 model_args로 변경
args = model_args

logger = logging.getLogger()

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

if training_args.should_log:
    transformers.utils.logging.set_verbosity_info()  # log level을 INFO로 변경

log_level = training_args.get_process_log_level()

# 우리가 가지고 있는 logger와 HuggingFace의 logger의 log level 설정
logger.setLevel(log_level)
transformers.utils.logging.set_verbosity(log_level)

# 기타 HuggingFace logger option들을 설정
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

logger.info(f"Training/evaluation parameters {training_args}")

config = AutoConfig.from_pretrained(args.model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(
    args.model_name_or_path, config=config, torch_dtype=args.torch_dtype
)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"

embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))




<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: teacher-ssamko (teacher-ssamko-familywave) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
INFO:root:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
deb

...

[INFO|tokenization_utils_base.py:2030] 2025-02-06 14:45:50,973 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6/vocab.json
[INFO|tokenization_utils_base.py:2030] 2025-02-06 14:45:50,974 >> loading file merges.txt from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6/merges.txt
[INFO|tokenization_utils_base.py:2030] 2025-02-06 14:45:50,976 >> loading file tokenizer.json from cache at None
[INFO|tokenization_utils_base.py:2030] 2025-02-06 14:45:50,977 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2030] 2025-02-06 14:45:50,978 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6/special_tokens_map.json
[INFO|tokenization_utils_base.py:2030] 2025-02-06 14:45:50,980 >> loading file 

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

[INFO|modeling_utils.py:3953] 2025-02-06 14:45:53,201 >> loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6/pytorch_model.bin
[INFO|modeling_utils.py:1641] 2025-02-06 14:45:53,236 >> Instantiating OPTForCausalLM model under default dtype torch.float16.
[INFO|configuration_utils.py:1140] 2025-02-06 14:45:53,240 >> Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1
}

[INFO|modeling_utils.py:4849] 2025-02-06 14:45:53,306 >> All model checkpoint weights were used when initializing OPTForCausalLM.

[INFO|modeling_utils.py:4857] 2025-02-06 14:45:53,308 >> All the weights of OPTForCausalLM were initialized from the model checkpoint at facebook/opt-125m.
If your task is similar to the task the model of the checkpoint was trained on, you can already use OPTForCausalLM for predictions without further training.
[INFO|safetensors_conversion.py:

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

[INFO|configuration_utils.py:1095] 2025-02-06 14:45:53,530 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6/generation_config.json
[INFO|configuration_utils.py:1140] 2025-02-06 14:45:53,532 >> Generate config GenerationConfig {
  "bos_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1
}



In [4]:
df = pd.read_csv("Enhanced_Instruction_Tuning_Dataset.csv")

# 데이터 샘플 확인
print(df.head())

# 컬럼 이름 확인
print(df.columns)

df.rename(columns={"Instruction": "instruction", "Input": "input", "Output": "output"}, inplace=True)

print(df.columns)

                Instruction               Input  \
0        상품이 불량이에요. 교환해주세요.        주문번호: 123456   
1       배송이 너무 늦어요! 언제 오나요?        주문번호: 789012   
2   환불해 주세요. 제품이 마음에 안 들어요.        주문번호: 345678   
3  앱에서 결제가 안 돼요. 계속 오류가 나요.  결제 오류 메시지: '승인 실패'   
4              회원 탈퇴하고 싶어요.       아이디: user1234   

                                              Output  
0  불편을 드려 죄송합니다. 주문번호 123456 확인 후 빠르게 교환 도와드리겠습니다...  
1  배송이 지연되어 죄송합니다. 주문번호 789012를 확인해 보니 현재 배송 중이며,...  
2  고객님, 만족스럽지 못하셨다니 죄송합니다. 주문번호 345678 확인 후 환불 절차...  
3  불편을 드려 죄송합니다. 오류 메시지를 확인해 보니 승인 과정에서 문제가 발생한 것...  
4  정말 아쉽습니다. 고객님의 의견을 반영하여 더 나은 서비스를 제공하도록 하겠습니다....  
Index(['Instruction', 'Input', 'Output'], dtype='object')
Index(['instruction', 'input', 'output'], dtype='object')


In [5]:
# 학습 데이터 준비
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Pandas DataFrame을 Hugging Face Dataset으로 변환
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# DatasetDict로 묶기 (Trainer에서 사용하기 좋음)
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

datasets.utils.logging.set_verbosity(log_level)


column_names = list(dataset["train"].features)
text_column_name = "text" if "text" in column_names else column_names[0]


def tokenize_function(examples):
    output = tokenizer(examples[text_column_name])
    return output


with training_args.main_process_first(desc="dataset map tokenization"):
    tokenized_datasets = dataset.map(
        tokenize_function,
        batched=True,
        num_proc=args.num_workers,
        remove_columns=column_names,
    )

max_pos_embeddings = (
    config.max_position_embeddings
    if hasattr(config, "max_position_embeddings")
    else 1024
)
block_size = (
    args.block_size
    if tokenizer.model_max_length is None
    else min(args.block_size, tokenizer.model_max_length)
)


def group_texts(examples):
    # 주어진 text들을 모두 concat 해줍니다.
    # 예를 들어 examples = {'train': [['Hello!'], ['Yes, that is great!']]}이면 결과물은 {'train': ['Hello! Yes, that is great!']}가 됩니다.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}

    # 전체 길이를 측정합니다.
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size

    # block_size로 text를 쪼갭니다.
    # 예를 들어 block_size=3일 때 {'train': ['Hello! Yes, that is great!']}는
    # {'train': ['Hel', 'lo!', ' Ye', 's, ', 'tha', ...]}가 됩니다.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }

    # Next token prediction이니 label은 자기 자신으로 설정합니다.
    result["labels"] = result["input_ids"].copy()
    return result


with training_args.main_process_first(desc="grouping texts together"):
    lm_datasets = tokenized_datasets.map(
        group_texts, batched=True, num_proc=args.num_workers
    )

split_datasets = lm_datasets["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]



Spawning 2 processes
INFO:datasets.arrow_dataset:Spawning 2 processes


Map (num_proc=2):   0%|          | 0/84 [00:00<?, ? examples/s]

Concatenating 2 shards
INFO:datasets.arrow_dataset:Concatenating 2 shards
Spawning 2 processes
INFO:datasets.arrow_dataset:Spawning 2 processes


Map (num_proc=2):   0%|          | 0/21 [00:00<?, ? examples/s]

Concatenating 2 shards
INFO:datasets.arrow_dataset:Concatenating 2 shards
Spawning 2 processes
INFO:datasets.arrow_dataset:Spawning 2 processes


Map (num_proc=2):   0%|          | 0/84 [00:00<?, ? examples/s]

Concatenating 2 shards
INFO:datasets.arrow_dataset:Concatenating 2 shards
Spawning 2 processes
INFO:datasets.arrow_dataset:Spawning 2 processes


Map (num_proc=2):   0%|          | 0/21 [00:00<?, ? examples/s]

Concatenating 2 shards
INFO:datasets.arrow_dataset:Concatenating 2 shards


In [6]:
metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  results = metric.compute(predictions=logits, references=labels)
  del logits, labels
  return results


trainer = SFTTrainer(
    model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    args=SFTConfig(output_dir="/tmp/clm-instruction-tuning"),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

# checkpoint = None
# last_checkpoint = get_last_checkpoint(
#     training_args.output_dir
# )  # 만약 output_dir에 checkpoint가 남아있으면 이를 사용하고, 없으면 None이 return됩니다.
# if (
#     training_args.resume_from_checkpoint is not None
# ):  # output_dir이 아닌 다른 위치에서의 checkpoint를 resume_from_checkpoint로 지정할 수 있습니다.
#     checkpoint = training_args.resume_from_checkpoint
# else:  # 아니면 last_checkpoint로 checkpoint를 지정합니다.
#     checkpoint = last_checkpoint

torch.cuda.empty_cache()
train_result = trainer.train()

trainer.save_model()

metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

if eval_dataset is not None:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate()
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)



[INFO|training_args.py:2176] 2025-02-06 14:48:46,514 >> PyTorch: setting up devices
[INFO|training_args.py:1851] 2025-02-06 14:48:46,536 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|configuration_utils.py:695] 2025-02-06 14:48:46,630 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--opt-125m/snapshots/27dcfa74d334bc871f3234de431e71c6eeba5dd6/config.json
[INFO|configuration_utils.py:762] 2025-02-06 14:48:46,632 >> Model config OPTConfig {
  "_name_or_path": "facebook/opt-125m",
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dro

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

[INFO|trainer.py:2362] 2025-02-06 14:48:47,691 >> ***** Running training *****
[INFO|trainer.py:2363] 2025-02-06 14:48:47,692 >>   Num examples = 84
[INFO|trainer.py:2364] 2025-02-06 14:48:47,694 >>   Num Epochs = 3
[INFO|trainer.py:2365] 2025-02-06 14:48:47,695 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:2368] 2025-02-06 14:48:47,696 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:2369] 2025-02-06 14:48:47,698 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2370] 2025-02-06 14:48:47,699 >>   Total optimization steps = 33
[INFO|trainer.py:2371] 2025-02-06 14:48:47,701 >>   Number of trainable parameters = 125,239,296
[INFO|integration_utils.py:811] 2025-02-06 14:48:47,707 >> Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different

Step,Training Loss


[INFO|trainer.py:3887] 2025-02-06 14:48:50,947 >> Saving model checkpoint to /tmp/clm-instruction-tuning/checkpoint-33
[INFO|configuration_utils.py:419] 2025-02-06 14:48:50,950 >> Configuration saved in /tmp/clm-instruction-tuning/checkpoint-33/config.json
[INFO|configuration_utils.py:909] 2025-02-06 14:48:50,952 >> Configuration saved in /tmp/clm-instruction-tuning/checkpoint-33/generation_config.json
[INFO|modeling_utils.py:3042] 2025-02-06 14:48:51,497 >> Model weights saved in /tmp/clm-instruction-tuning/checkpoint-33/model.safetensors
[INFO|tokenization_utils_base.py:2485] 2025-02-06 14:48:51,501 >> tokenizer config file saved in /tmp/clm-instruction-tuning/checkpoint-33/tokenizer_config.json
[INFO|tokenization_utils_base.py:2494] 2025-02-06 14:48:51,503 >> Special tokens file saved in /tmp/clm-instruction-tuning/checkpoint-33/special_tokens_map.json
[INFO|trainer.py:2636] 2025-02-06 14:48:52,340 >> 

Training completed. Do not forget to share your model on huggingface.co/models =

***** train metrics *****
  epoch                    =        3.0
  total_flos               =    20102GF
  train_loss               =     1.0595
  train_runtime            = 0:00:04.64
  train_samples_per_second =       54.3
  train_steps_per_second   =      7.111


ValueError: Module inputs don't match the expected format.
Expected format: {'predictions': Value(dtype='string', id=None)},
Input predictions: [[[  nan   nan   nan ...   nan   nan   nan]
  [  nan   nan   nan ...   nan   nan   nan]
  [  nan   nan   nan ...   nan   nan   nan]
  ...
  [-100. -100. -100. ... -100. -100. -100.]
  [-100. -100. -100. ... -100. -100. -100.]
  [-100. -100. -100. ... -100. -100. -100.]]

 [[  nan   nan   nan ...   nan   nan   nan]
  [  nan   nan   nan ...   nan   nan   nan]
  [  nan   nan   nan ...   nan   nan   nan]
  ...
  [-100. -100. -100. ... -100. -100. -100.]
  [-100. -100. -100. ... -100. -100. -100.]
  [-100. -100. -100. ... -100. -100. -100.]]

 [[  nan   nan   nan ...   nan   nan   nan]
  [  nan   nan   nan ...   nan   nan   nan]
  [  nan   nan   nan ...   nan   nan   nan]
  ...
  [-100. -100. -100. ... -100. -100. -100.]
  [-100. -100. -100. ... -100. -100. -100.]
  [-100. -100. -100. ... -100. -100. -100.]]

 ...

 [[  nan   nan   nan ...   nan   n